In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff  # 圖形工廠
from plotly.subplots import make_subplots  # 繪製子圖
import numpy as np
from Upload_git import upload
from os import system
import time
from time import ctime, sleep
from Upload_git import upload
import schedule
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from statsmodels.regression.rolling import RollingOLS, RollingWLS
from scipy import stats
import revenue_function as fn
import pickle

In [3]:
"""
connect to db
"""
host='database-1.cyn7ldoposru.us-east-1.rds.amazonaws.com'
port='5432'
user='Yu'
password='m#WA12J#'
database="JQC_Revenue1"

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
        user, password, host, port, database), echo=True)

"""
callback input & output detail 
"""
revenue = pd.read_sql('industry', engine)
new_industry_name = revenue['TSE新產業名.1'].dropna()
minor_industry_name = revenue['TEJ子產業名.1'].dropna()
db = pd.read_sql('tej_revenue', engine).drop('index', axis=1)
db['rev'] = round(db['rev']/1000, 2)
stock_info = revenue.iloc[:, 1:4].rename({'TSE新產業名':'new_industry_name',
                                          'TEJ子產業名':'minor_industry_name'}, axis=1)
stock_info['st_code'] = stock_info['公司簡稱'].str[0:4]
stock_info['st_name'] = stock_info['公司簡稱'].str[5:]
stock_info = stock_info.drop('公司簡稱', axis=1)

2023-03-08 23:10:22,713 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-08 23:10:22,714 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-08 23:10:23,134 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-08 23:10:23,134 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-08 23:10:23,554 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-08 23:10:23,555 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-08 23:10:23,976 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-03-08 23:10:23,977 INFO sqlalchemy.engine.Engine [generated in 0.00073s] {'name': 'industry'}
2023-03-08 23:10:24,606 INFO sqlalchemy.engine.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2023-03-08 23:10:24,608 INFO sqlalchemy.engine.Engine [generated in 0

2023-03-08 23:10:26,946 INFO sqlalchemy.engine.Engine [generated in 0.00078s] {'table_oid': 16461}
2023-03-08 23:10:27,156 INFO sqlalchemy.engine.Engine 
            SELECT
                pgd.description as table_comment
            FROM
                pg_catalog.pg_description pgd
            WHERE
                pgd.objsubid = 0 AND
                pgd.objoid = %(table_oid)s
        
2023-03-08 23:10:27,156 INFO sqlalchemy.engine.Engine [generated in 0.00065s] {'table_oid': 16461}
2023-03-08 23:10:27,585 INFO sqlalchemy.engine.Engine SELECT industry.index, industry."公司簡稱", industry."TSE新產業名", industry."TEJ子產業名", industry."Unnamed: 3", industry."TSE新產業名.1", industry."TEJ子產業名.1", industry."Unnamed: 6", industry."公司簡稱.1", industry."TSE新產業名.2", industry."TEJ子產業名.2", industry."Unnamed: 10", industry."Unnamed: 11", industry."單月營收(千元)", industry."單月營收(千元).1", industry."單月營收(千元).2", industry."單月營收(千元).3", industry."單月營收(千元).4", industry."單月營收(千元).5", industry."單月營收(千元).6", industry."單月營收(

2023-03-08 23:10:32,188 INFO sqlalchemy.engine.Engine [cached since 5.667s ago] {'table_oid': 16512}
2023-03-08 23:10:32,402 INFO sqlalchemy.engine.Engine 
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        
2023-03-08 23:10:32,403 INFO sqlalchemy.engine.Engine [cached since 5.669s ago] {'table_oid': 16512}
2023-03-08 23:10:32,613 INFO sqlalchemy.engine.Engine 
            SELECT
                cons.conname as name,
                pg_get_constraintdef(cons.oid) as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                c

In [4]:
def get_yoy(data):
    data = data.sort_values('Year')
    data['yoy'] = round(data.rev.diff()/data.rev.shift(1), 4)
    return data


def get_st_mom_yoy(data):
    data['Year'] =  data.rev_period.astype(str).str[0:4]
    data['month'] = data.rev_period.astype(str).str[5:7]
    data['mom'] = round(data.rev.diff()/data.rev.shift(1), 4)
    yoy = pd.DataFrame()
    yoy = data.groupby('month').apply(get_yoy).reset_index(drop=True, inplace=False)
    return yoy#[yoy.Year >= '2016']


def get_ratio(data):
    ttl_rev = data.rev.sum()/len(data)*12
    data['ratio'] = round(data.rev/ttl_rev, 4)
    return data


def get_avg_ratio(data):
    if len(data) >= 5:
        weight = [1, 1, 1, 3, 5]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 4:
        weight = [1, 1, 2, 3]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 3:
        weight = [1, 1, 2]
        roll_num = len(weight)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(np.average(data.ratio, weights = weight), 4)
    
    elif len(data) >= 2:
        roll_num = len(data)
        data = data.sort_values('Year').reset_index(drop=True, inplace=False).tail(roll_num + 1).head(roll_num)  # 抓到最後一年的前 n 年，n 隨權重年份個變
        return round(data.ratio.mean(), 4)

# def get_predict(data):
#     new_industry_list = ['M1722 生技醫療', 'M2500 建材營造', 'M9900 其他', 'M1200 食品工業']
#     minor_industry_list = ['M25A 建設']
#     if (data.new_industry_name.head(1).isin(new_industry_list).values[0] == False) & (data.new_industry_name.head(1).isin(minor_industry_list).values[0] == False):
#         if data.shape[0] >= 10:
#             a = data.groupby('Year', as_index=False).apply(get_ratio)
#             ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
#             predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(13)  # 僅向後預測一年
#             predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
#             predict['predict_month_rev'] = round(predict['predict_annul_rev'].tail(12).mean()* predict.wt_avg_ratio)
#             predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
#             predict['rev'] = predict['predict_month_rev']
#             predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
#             predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
#             predict = predict.groupby('month', as_index=False).apply(get_yoy).sort_values(['Year', 'month'])
#             predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
#             predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
#             return predict.tail(12)
#         else:
#             print(data.st_code.head(1).values[0], data.shape[0])
#     else:
#         print(data.st_code.head(1).values[0], data.new_industry_name.head(1).values[0], '產業不可測')

        

def get_predict(data):
    # new_industry_list = ['M1722 生技醫療', 'M2500 建材營造', 'M9900 其他', 'M1200 食品工業']
    # minor_industry_list = ['M25A 建設']
    # if (data.new_industry_name.head(1).isin(new_industry_list).values[0] == False) & (data.new_industry_name.head(1).isin(minor_industry_list).values[0] == False):
    if data.shape[0] >= 10:
        a = data.groupby('Year', as_index=False).apply(get_ratio)
        ratio = pd.DataFrame(a.groupby('month', as_index=False).apply(get_avg_ratio)).set_axis(['month', 'wt_avg_ratio'],axis=1)
        predict = pd.merge(a, ratio, on='month', how='outer').sort_values('rev_period').reset_index(drop=True,inplace=False).tail(12)  # 僅向後預測一年
        predict['predict_annul_rev'] = round(predict.rev / predict.ratio)
        predict['predict_month_rev'] = round(predict['predict_annul_rev'].tail(12).mean()* predict.wt_avg_ratio)
        predict['Year'] = (predict.Year.astype(int) + 1).astype(str)
        predict['rev'] = predict['predict_month_rev']
        predict = pd.concat([data.sort_values('rev_period'), predict], axis=0).sort_values(['Year', 'month'])
        predict['mom'] = predict.rev.diff() / predict.rev.shift(1)
        predict = predict.groupby('month', as_index=False).apply(fn.get_yoy).sort_values(['Year', 'month'])
        predict['rev_period'] = '* ' + predict.Year + '/' + predict.month
        predict = predict.drop(['ratio','wt_avg_ratio','predict_annul_rev','predict_month_rev'], axis=1)
        return predict.tail(12)
    else:
        print(data.st_code.head(1).values[0], data.shape[0])
    # else:
    #     print(data.st_code.head(1).values[0], data.new_industry_name.head(1).values[0], '產業不可測')


def growth(data):
    data['yoy年成長大於10'] = np.where(data.yoy >= 10, 1, 0)
    data['yoy成長'] = np.where(data.yoy > data.yoy.shift(1), 1, 0)
    data['yoy衰退'] = np.where(data.yoy <= data.yoy.shift(1), 1, 0)
    
    test = data.loc[:, ['yoy年成長大於10']]
    data['yoy近五月大於10'] = test.rolling(5).sum()
    data['yoy連續大於10'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    
    test = data.loc[:, ['yoy成長']]
    data['yoy近五月成長'] = test.rolling(5).sum()
    data['yoy連續成長'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    
    test = data.loc[:, ['yoy衰退']]
    data['yoy近五月衰退'] = test.rolling(5).sum()
    data['yoy連續衰退'] = test.apply(lambda x: x.groupby((x != x.shift()).cumsum()).cumsum())
    return data


def get_mompic(st_data, st_predict, pic_title):
    if st_predict.shape[0] != 0:
        MOM_pic = []
        for i in st_data.Year.drop_duplicates():
            globals()['trace' + str(i)] = go.Scatter(x=st_data[st_data.Year == i].month, y=st_data[st_data.Year == i].mom, name=i, opacity=0.5)
            MOM_pic.append(globals()['trace' + str(i)])
        for i in st_predict.Year.drop_duplicates():
            globals()['trace_predict' + str(i)] = go.Scatter(x=st_predict[st_predict.Year == i].month, y=st_predict[st_predict.Year == i].mom, name=str(i) + '預估值', #marker_color='black',
                                                             line=dict(dash = "dash"), connectgaps=False)
            MOM_pic.append(globals()['trace_predict' + str(i)])
        layout = go.Layout(title=pic_title+' MOM %', xaxis_title="月份", yaxis_title="MOM %", hovermode='x unified', legend_title_text='年份')
        MOM = go.Figure(data = MOM_pic, layout=layout)
    else:
        MOM_pic = []
        for i in st_data.Year.drop_duplicates():
            globals()['trace' + str(i)] = go.Scatter(x=st_data[st_data.Year == i].month, y=st_data[st_data.Year == i].mom, name=i, opacity=0.5)
            MOM_pic.append(globals()['trace' + str(i)])
        layout = go.Layout(title=pic_title+' MOM %', xaxis_title="月份", yaxis_title="MOM %", hovermode='x unified', legend_title_text='年份')
        MOM = go.Figure(data = MOM_pic, layout=layout)
    return MOM


def get_yoypic(st_yoy, st_predict, pic_title):
    if st_predict.shape[0] != 0:
        YOY_pic = []
        for i in st_yoy.Year.drop_duplicates():
            globals()['trace' + str(i)] = go.Scatter(x=st_yoy[st_yoy.Year == i].month, y=st_yoy[st_yoy.Year == i].yoy, name=i, opacity=0.5)
            YOY_pic.append(globals()['trace' + str(i)])
        for i in st_predict.Year.drop_duplicates():
            globals()['trace_predict' + str(i)] = go.Scatter(x=st_predict[st_predict.Year == i].month, y=st_predict[st_predict.Year == i].yoy, name=str(i) + '預估值',  #marker_color='black',
                                                             line=dict(dash = "dash"), connectgaps=False)
            YOY_pic.append(globals()['trace_predict' + str(i)])
        layout = go.Layout(title=pic_title+' YOY %', xaxis_title="月份", yaxis_title="YOY %", hovermode='x unified', legend_title_text='年份')
        YOY = go.Figure(data = YOY_pic, layout=layout)
    else:
        YOY_pic = []
        for i in st_yoy.Year.drop_duplicates():
            globals()['trace' + str(i)] = go.Scatter(x=st_yoy[st_yoy.Year == i].month, y=st_yoy[st_yoy.Year == i].yoy, name=i, opacity=0.5)
            YOY_pic.append(globals()['trace' + str(i)])
        layout = go.Layout(title=pic_title+' YOY %', xaxis_title="月份", yaxis_title="YOY %", hovermode='x unified', legend_title_text='年份')
        YOY = go.Figure(data = YOY_pic, layout=layout)
    return YOY


def get_revpic(st_data, st_predict, title_name):
    if st_predict.shape[0] != 0:
        REV_pic = []
        for i in st_data.Year.drop_duplicates():
            globals()['trace' + str(i)] = go.Bar(x=st_data[st_data.Year == i].month, y=st_data[st_data.Year == i].rev, name=i, opacity=0.7)
            REV_pic.append(globals()['trace' + str(i)])
        trace_predict = go.Bar(x=st_predict.month, y=st_predict.predict_month_rev, opacity=1,
                               marker_color='black', name='預估值', marker_pattern_shape="\\",
                               marker=dict(color="white", line_color="black", pattern_fillmode="replace"))
        REV_pic.append(trace_predict)
        layout = go.Layout(barmode = 'group', title=title_name, xaxis_title="月份", yaxis_title="累計營收", hovermode='x unified', legend_title_text='年份')
        REV = go.Figure(data = REV_pic, layout=layout)
    else:
        REV_pic = []
        for i in st_data.Year.drop_duplicates():
            globals()['trace' + str(i)] = go.Bar(x=st_data[st_data.Year == i].month, y=st_data[st_data.Year == i].rev, name=i, opacity=0.7)
            REV_pic.append(globals()['trace' + str(i)])
        layout = go.Layout(barmode = 'group', title=title_name, xaxis_title="月份", yaxis_title="累計營收", hovermode='x unified', legend_title_text='年份')
        REV = go.Figure(data = REV_pic, layout=layout)
    return REV


#### 模擬查詢結果

In [55]:
dropdown1 = new_industry_name[2]
dropdown2 = minor_industry_name[5]
st_input = '4763'
st_or_gruop = '個股'

In [56]:
if st_input is None or st_input == '':
    try:
        if (dropdown1 != new_industry_name[0]) and (dropdown2 == minor_industry_name[0]):
            mask = ("new_industry_name = '" + dropdown1 + "'")

        elif (dropdown2 != minor_industry_name[0]) and (dropdown1 == new_industry_name[0]):
            mask = ("minor_industry_name = '" + dropdown2 + "'")

        elif (dropdown1 != new_industry_name[0]) and (dropdown2 != minor_industry_name[0]):
            mask = ("new_industry_name = '" + dropdown1 + "' and " + "minor_industry_name = '" + dropdown2 + "'")

        # 從 db 撈資料
        st_search = """
        select st_code, st_name, rev_period, declaration_date, rev
        from tej_revenue
        where {};
        """.format(mask)

        data = pd.read_sql(st_search, engine)
        data = data.groupby(by='st_code', as_index=False).apply(fn.get_st_mom_yoy).reset_index(drop=True,inplace=False)

        # 產業最新營收公布情形
        latest = fn.get_latest(data)

        # 產業總營收狀況
        data = data.groupby(by='rev_period', as_index=False).agg({'rev': 'sum'})
        data = fn.get_st_mom_yoy(data)

        # 預測資料
        data_predict = fn.predict(data)

        # 繪圖
        title = dropdown1 + ' '
        REV = fn.get_revpic(data, data_predict, (title + ' 各年度月營收'))
        MOM = fn.get_mompic(data, data_predict, (title + ' 各年度月營收'))
        YOY = fn.get_yoypic(data, data_predict, (title + ' 各年度月營收'))

        # 整理營收資訊表
        table = data.pivot_table(values='rev', index='Year', columns='month').reset_index()
        data_predict.Year = data_predict.Year + str(' 預估值')
        table = pd.concat([table, data_predict.tail(12).pivot_table(index='Year', columns='month', values='rev').reset_index()],axis=0)
        table = table.to_dict('records')
#         return table, latest, REV, MOM, YOY

    except KeyError:
        print("跳警示通知：該分類沒有相關個股，請重新選擇，並reset")


else:
    try:
        if (st_input.isdigit() == True):
            industry = stock_info[stock_info.st_code == st_input].new_industry_name.values[0]
            mask = ("new_industry_name = '" + industry + "'")
            st_mask = 'st_code'

        else:
            industry = stock_info[stock_info.st_name == st_input].new_industry_name.values[0]
            mask = ("new_industry_name = '" + industry + "'")
            st_mask = 'st_name'

        # 從 db 撈資料
        st_search = """
        select st_code, st_name, rev_period, declaration_date, rev
        from tej_revenue
        where {};
        """.format(mask)
        data = pd.read_sql(st_search, engine)
        data = data.groupby(by='st_code', as_index=False).apply(fn.get_st_mom_yoy).reset_index(drop=True,inplace=False)

        # 產業最新營收公布情形
        latest = fn.get_latest(data)

        # 轉換成個股總營收
        data = data[data[st_mask] == st_input]
        data = fn.get_st_mom_yoy(data)

        # 預測資料
        data_predict = fn.predict(data)

        # 繪圖
        title = data.st_code[0] + ' ' + data.st_name[0]
        REV = fn.get_revpic(data, data_predict, (title + ' 各年度月營收'))
        MOM = fn.get_mompic(data, data_predict, (title + ' 各年度月營收'))
        YOY = fn.get_yoypic(data, data_predict, (title + ' 各年度月營收'))

        # 整理營收資訊表
        table = data.pivot_table(values='rev', index='Year', columns='month').reset_index()
        data_predict.Year = data_predict.Year + str(' 預估值')
        table = pd.concat([table, data_predict.tail(12).pivot_table(index='Year', columns='month', values='rev').reset_index()],axis=0)
        table = table.to_dict('records')
#         return table, latest, REV, MOM, YOY

    except IndexError:
        print('跳警示通知：查無該股票營收，請重新輸入，並reset')

2023-03-07 12:32:07,623 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2023-03-07 12:32:07,625 INFO sqlalchemy.engine.Engine [cached since 2951s ago] {'name': "\n        select st_code, st_name, rev_period, declaration_date, rev\n        from tej_revenue\n        where new_industry_name = 'M1721 化學工業';\n        "}
2023-03-07 12:32:08,242 INFO sqlalchemy.engine.Engine 
        select st_code, st_name, rev_period, declaration_date, rev
        from tej_revenue
        where new_industry_name = 'M1721 化學工業';
        
2023-03-07 12:32:08,243 INFO sqlalchemy.engine.Engine [raw sql] {}


In [58]:
data = db.groupby('st_code', as_index=False).apply(get_st_mom_yoy)
data = data.reset_index(drop=True)
# predict = data.groupby('st_code', as_index=False).apply(get_predict)
# predict = predict.reset_index(drop=True)

In [ ]:
data = pd.concat([data, predict], axis=0)#.sort_values(['st_code', 'rev_period'], axis=0)

#### 模擬查詢結果

In [71]:
dropdown1 = new_industry_name[2]
dropdown2 = minor_industry_name[5]
st_input = ''
st_or_gruop = '個股'


# def revenue(dropdown1, dropdown2, st_input, st_or_gruop):
if (st_input == ''):
    if (dropdown1 != new_industry_name[0]) & (dropdown2 == minor_industry_name[0]):
        rev_data = data[(data.new_industry_name == dropdown1)]
        title = dropdown1 + ' '

    elif (dropdown1 != new_industry_name[0]) & (dropdown2 != minor_industry_name[0]):
        rev_data = data[(data.new_industry_name == dropdown1) & 
                        (data.minor_industry_name == dropdown2)]
        title = dropdown1 + dropdown2 + ' '

    elif (dropdown1 == new_industry_name[0]) & (dropdown2 != minor_industry_name[0]):
        rev_data = data[(data.minor_industry_name == dropdown2)]
        title = dropdown2 + ' '
        
    rev_data = rev_data.groupby('rev_period', as_index=False).agg({'rev':'sum'}).reset_index(drop=True)
    rev_data = get_st_mom_yoy(rev_data)


else:
    if st_or_gruop == '個股':
        rev_data = data[data.st_code == st_input]
        title = st_input + ' ' + rev_data.st_name.head(1).values[0] + ' '

    elif st_or_gruop == '產業':
        new_index = data[data.st_code == st_input].new_industry_name.head(1).values[0]
        minor_index = data[data.st_code == st_input].minor_industry_name.head(1).values[0]
        if (new_index != '') & (minor_index != ''):
            rev_data = data[(data.new_industry_name == new_index) 
                            & (data.minor_industry_name == minor_index)]
            title = new_index + minor_index + ' '

        elif (new_index != '') & (minor_index == ''):
            rev_data = data[(data.new_industry_name == new_index)]
            title = new_index + ' '

        elif (new_index == '') & (minor_index != ''):
            rev_data = data[(data.minor_industry_name == minor_index)]
            title = minor_index + ' '
    rev_data = rev_data.drop(['declaration_date', 'declaration_year','declaration_month','new_industry_name','minor_industry_name'], axis=1)

rev_predict = get_predict(rev_data)
rev_data
rev_predict

# pd.concat([rev_data, get_predict(rev_data)], axis=0).reset_index(drop=True).tail(25)
# revenue(dropdown1, dropdown2, st_input, st_or_gruop)#.drop_duplicates('st_code')

,rev_period,rev,Year,month,mom,yoy
108,2022/01,21403.65,2022,01,0.046500,0.1760
109,2022/02,15959.99,2022,02,-0.254300,0.1834
110,2022/03,20120.64,2022,03,0.260700,0.1218
111,2022/04,19675.36,2022,04,-0.022100,0.0517
112,2022/05,19480.76,2022,05,-0.009900,0.0904
113,2022/06,21695.70,2022,06,0.113700,0.1505
114,2022/07,19813.60,2022,07,-0.086700,0.0530
115,2022/08,21761.33,2022,08,0.098300,0.2442
116,2022/09,21010.94,2022,09,-0.034500,0.0942
117,2022/10,21143.00,2022,10,0.006300,0.1567


In [8]:
r = '/Users/yuchun/Revenue/ML_Cluster/'
industry = pd.read_excel(r+'stock_info.xls', sheet_name='產業別').loc[
    :, ['代號','名稱','交易所主產業代碼','交易所子產業代碼','TEJ主產業代碼','TEJ子產業代碼']]

new_industry = pd.read_excel(r+'stock_info.xls', sheet_name='主產業').drop('產業英文簡稱', axis=1)
new_industry['TEJ主產業代碼'] = new_industry['TEJ主產業代碼'].str.strip().str[1:]
minor_industry = pd.read_excel(r+'stock_info.xls', sheet_name='子產業').drop('產業英文簡稱', axis=1)
minor_industry['TEJ子產業代碼'] = minor_industry['TEJ子產業代碼'].str.strip().str[1:]

industry = pd.merge(industry, new_industry, on='TEJ主產業代碼', how='left')
industry = pd.merge(industry, minor_industry, on='TEJ子產業代碼', how='left')

industry['industry_title'] = np.where((industry.minor_industry.isna() == True) 
                                      & (industry.new_industry.isna() == False), 
                                     industry.new_industry + '類', 
                                      np.where((industry.minor_industry.isna() == False) 
                                               & (industry.new_industry.isna() == True), 
                                               industry.minor_industry + '類', 
                                               industry.new_industry + ':' + industry.minor_industry + '類'))
industry.to_csv('industry.csv', encoding='utf_8_sig', header=True)

In [ ]:
update = pd.read_excel("2022月營收更新報表_yuchun.xlsm", sheet_name='工作表1', header=6)
update = update.iloc[:, 3:7]
update = update.rename({'【1】年月':'rev_period', '【2】營收發布日':'declaration_date', '【3】單月營收(千元)':'rev'}, axis=1)
update['st_code'] = update['代號 名稱'].astype(str).str[0:4]
update['st_name'] = update['代號 名稱'].astype(str).str[5:]
update['declaration_date'] = update['declaration_date'].astype(str)
update['declaration_year'] = update['declaration_date'].str[0:4]
update['declaration_month'] = update['declaration_date'].str[4:6]
update['rev_period'] = update['rev_period'].astype(str).str[0:4] + "/" + update['rev_period'].astype(str).str[4:6]
update['declaration_date'] = update['declaration_date'].str[0:4] + "/" + update['declaration_date'].str[4:6] + "/" + update['declaration_date'].str[6:8]
update = update[update.declaration_date != 'nan//'].drop('代號 名稱', axis=1)
update.reset_index(drop=True)

,rev_period,declaration_date,rev,st_code,st_name,declaration_year,declaration_month
0,2023/02,2023/03/03,60985.0,1437,勤益控,2023,03
1,2023/02,2023/03/02,84182.0,1452,宏益,2023,03
2,2023/02,2023/03/02,2653731.0,1477,聚陽,2023,03
3,2023/02,2023/03/03,516402.0,1560,中砂,2023,03
4,2023/02,2023/03/03,153690.0,1586,和勤,2023,03
...,...,...,...,...,...,...,...
114,2023/02,2023/03/05,561018.0,8436,大江,2023,03
115,2023/02,2023/03/03,70181.0,8440,綠電,2023,03
116,2023/02,2023/03/02,511341.0,8924,大田,2023,03
117,2023/02,2023/03/03,114625.0,8940,新天地,2023,03
